In [3]:
from dgp import *

# 1. Simulate or load data
adj = get_graph(800, 1, 2, seed=1)

tau = np.array([-1.0, 0.50, -0.50])       # shape (3,)
rho = np.array([[0,0.1,0.2],
                [0.1,0,0.1],
                [0.2,0.1,0]])      # shape (3, 3), with 0s on the diagonal
nu = np.array([0.1,0,0,0.1,0,0,0.1,0,0]).reshape(3,3)       # shape (3, 3)
gamma = np.array([-1.00,0.50,0.10,0.20,0.05,0.25,-0.08,0.30])     # shape (8,)
beta = np.array([-0.30,-0.60,-0.20,-0.20,-0.05,-0.10,-0.01,0.40,0.01,0.20])     # shape (10,)

Y_chain, A_chain, L_chain = sample_network_chain(adj, tau, rho, nu, gamma, beta, R=500,
    burnin_R=20, seed=0)

Y_chain = Y_chain[::5]
A_chain = A_chain[::5]
L_chain = L_chain[::5]

100%|██████████| 520/520 [00:43<00:00, 12.07it/s]


In [4]:
from agcEffect import *

agc_effect(
    adj,
    tau, rho, nu, beta,
    treatment_allocation=0.7,
    R=200,
    burnin_R=50,
    seed=0
)

{'average': 0.31335625,
 'direct_effect': -0.14082500000000003,
 'spillover_effect': -0.05294999999999994,
 'psi_gamma': 0.31335625,
 'psi_1_gamma': 0.27003750000000004,
 'psi_0_gamma': 0.4108625000000001,
 'psi_zero': 0.4638125}

In [ ]:
from utils import run_pll
from drnet import doubly_robust

def run_dr(A_chain, L_chain, Y_chain, adj, i):
    """
    Run doubly robust estimator
    """
    ret_i = doubly_robust(A_chain[i], L_chain[i], Y_chain[i], adj)
    
    return (ret_i['average'], ret_i['direct_effect'], ['spillover_effect'], 
            ret_i['psi_0_gamma'], ret_i['psi_zero'], ret_i['psi_1_gamma'], ret_i['psi_zero'])

args = [{'Y_chain': Y_chain[i],
         'A_chain': A_chain[i],
         'L_chain': L_chain[i],
         'adj': adj,
         'i': i} for i in range(len(Y_chain[:8]))]
n_cpu = 4
res_list = run_pll(run_dr, args, processes=n_cpu)

Multiprocessing <function run_dr at 0x7ff00a690b80> in 8 tasks, with 4 processes...


In [ ]:
res_list

[]

In [ ]:
from drnet import doubly_robust

ret = {
    'average': [],
    'direct_effect': [],
    'spillover_effect': [],
    'psi_0_gamma': [],
    'psi_zero': [],
    "psi_1_gamma":[],
}

for Y, A, L in tqdm(zip(Y_chain, A_chain, L_chain)):
    ret_i = doubly_robust(A, L, Y, adj, treatment_allocation=0.7, num_rep=1000)
    ret['average'].append(ret_i['average'])
    ret['direct_effect'].append(ret_i['direct_effect'])
    ret['spillover_effect'].append(ret_i['spillover_effect'])
    ret['psi_0_gamma'].append(ret_i['psi_0_gamma'])
    ret['psi_zero'].append(ret_i['psi_zero'])
    ret['psi_1_gamma'].append(ret_i['psi_1_gamma'])
    
print('Average effect:', np.mean(ret['average']))
print('Direct effect:', np.mean(ret['direct_effect']))
print('Spillover effect:', np.mean(ret['spillover_effect']))
print('Psi_0_gamma:', np.mean(ret['psi_0_gamma']))
print('Psi_zero:', np.mean(ret['psi_zero']))
print('Psi_1_gamma:', np.mean(ret['psi_1_gamma']))

1it [01:54, 114.73s/it]

psi_zero: [0.45084564742574423]
beta_hat: 0.453273978421918
psi_0_gamma: 0.4046804633460057


2it [03:44, 112.00s/it]

psi_zero: [0.4290952999162929]
beta_hat: 0.43190406753580407
psi_0_gamma: 0.3766375705823807


3it [05:41, 114.18s/it]

...

psi_zero: [0.460427697203892]
beta_hat: 0.46180241940865674
psi_0_gamma: 0.4124729516118059


37it [1:28:18, 162.74s/it]

psi_zero: [0.44124492405859816]
beta_hat: 0.45230150528319657
psi_0_gamma: 0.46111099747518575


37it [1:29:56, 145.84s/it]


KeyboardInterrupt: 

## Dense network

In [ ]:
from dgp import *

# 1. Simulate or load data
adj = get_graph(2000, 8, 10, seed=1)

tau = np.array([-1.0, 0.50, -0.50])       # shape (3,)
rho = np.array([[0,0.1,0.2],
                [0.1,0,0.1],
                [0.2,0.1,0]])      # shape (3, 3), with 0s on the diagonal
nu = np.array([0.1,0,0,0.1,0,0,0.1,0,0]).reshape(3,3)       # shape (3, 3)
gamma = np.array([-1.00,0.50,0.10,0.20,0.05,0.25,-0.08,0.30])     # shape (8,)
beta = np.array([-0.30,-0.60,-0.20,-0.20,-0.05,-0.10,-0.01,0.40,0.01,0.20])     # shape (10,)

Y_chain, A_chain, L_chain = sample_network_chain(adj, tau, rho, nu, gamma, beta, R=500,
    burnin_R=20, seed=0)

Y_chain = Y_chain[::5]
A_chain = A_chain[::5]
L_chain = L_chain[::5]

In [ ]:
from agcEffect import *

agc_effect(
    adj,
    tau, rho, nu, beta,
    treatment_allocation=0.7,
    R=200,
    burnin_R=50,
    seed=0
)

In [ ]:
from drnet import doubly_robust

ret = {
    'average': [],
    'direct_effect': [],
    'spillover_effect': [],
    'psi_0_gamma': [],
    'psi_zero': [],
    "psi_1_gamma":[],
}

for Y, A, L in tqdm(zip(Y_chain, A_chain, L_chain)):
    ret_i = doubly_robust(A, L, Y, adj, treatment_allocation=0.7, num_rep=1000)
    ret['average'].append(ret_i['average'])
    ret['direct_effect'].append(ret_i['direct_effect'])
    ret['spillover_effect'].append(ret_i['spillover_effect'])
    ret['psi_0_gamma'].append(ret_i['psi_0_gamma'])
    ret['psi_zero'].append(ret_i['psi_zero'])
    ret['psi_1_gamma'].append(ret_i['psi_1_gamma'])
    
print('Average effect:', np.mean(ret['average']))
print('Direct effect:', np.mean(ret['direct_effect']))
print('Spillover effect:', np.mean(ret['spillover_effect']))
print('Psi_0_gamma:', np.mean(ret['psi_0_gamma']))
print('Psi_zero:', np.mean(ret['psi_zero']))
print('Psi_1_gamma:', np.mean(ret['psi_1_gamma']))